In [1]:
%logstart -o -t

import sys

import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=4

#Sample Size
sampleN = 2000

import time

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
enable_rf_features = True
feature_all = False
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
enable_kmeans = True
enable_logistic_regression_l1 = True

# Enable Algorithms
enable_logistic_regression = True
enable_decision_tree = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_rf = True

#Multiclass classification, binary if falase
multiclass = True

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarn

In [2]:
print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if enable_rf_features:
    print("Feature Selection by RandomForest: Enabled")
else:
    print("Feature Selection by RandomForest: Disabled")
    print("Default Feature: {}".format(defaultFeatures))

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_logistic_regression:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_decision_tree:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")
    
if enable_rf:
    print("Random Forest: Enabeld")
else:
    print("Random Forest: Disabled")

Sample size: 2000
Multi-Class Classification: Enabled
Grid Search: Disabled
Feature Selection by RandomForest: Enabled
All Features: Disabled
K-means: Enabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
MLP: Enabled
Random Forest: Enabeld


In [3]:
#sys.stdout=open("Initial_Binary_Model2.out", "w")

In [4]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Sun Nov 11 14:45:29 2018


In [5]:
#df = pd.read_csv('data01_simple.csv', engine = 'python')
df = pd.read_csv('data01_clean.csv', engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0    1999       1       1      20       2      34       8       1       5   
1    1999       1       1      20       2      34       2       1       5   

   C_RALN   ...    V_ID  V_TYPE  V_YEAR  P_ID  P_SEX  P_AGE  P_PSN  P_SAFE  \
0       3   ...       1       6    1990     1      1     41     11       2   
1       3   ...       2       1    1987     1      1     19     11       2   

   P_USER  P_ISEV  
0       1       1  
1       1       1  

[2 rows x 21 columns]


In [6]:
print(df.isnull().sum().sum())

0


In [7]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [8]:
df_cat = df.astype('category').copy()

In [9]:
df_int = df.astype('int').copy()

In [10]:
print(df.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


## Convert Class Variable to Binary if multi class disabled

In [11]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
if multiclass:
    #Undersample the majority for the 3 class evaluation
    
    df_class = df_cat.copy()
    
    # subset fatal class
    is_fatal =  df_class['P_ISEV']==3
    is_fatal_count = is_fatal.sum()
    print("Number of Fatal: {}".format(is_fatal_count))
    df_class_fatal = df_class[is_fatal]
    print(df_class_fatal.head(2))
    
    # subset injury class
    is_injury =  df_class['P_ISEV']==2
    is_injury_count = is_injury.sum()
    print("Number of Injury: {}".format(is_injury_count))
    df_class_injury = df_class[is_injury]
    print(df_class_injury.head(2))
    
    # subset non_injury class
    is_safe =  df_class['P_ISEV']==1
    is_safe_count = is_safe.sum()
    print("Number of Non-Injury: {}".format(is_safe_count))
    df_class_safe = df_class[is_safe]
    print(df_class_safe.head(2))
    
    # get the size of fatal datafram
    min_size = df_class_fatal.index.size
    print("Size of Fatal Subset: {}".format(min_size))
    
    # get size of injury
    print("Size of injury Subset: {}".format(df_class_injury.index.size))
    
    # size of non-fatal
    print("Size of non-fatal Subset: {}".format(df_class_safe.index.size))
    
    # randomly sample n number of injury and no injury and append to fatal
    df_class_injury_select = df_class_injury.sample(n=min_size)
    print("Shape of injury sampled dataframe: {}".format(df_class_injury_select.shape))
    df_class_safe_select = df_class_safe.sample(n=min_size)
    print("Shape of nom-injury sampled dataframe: {}".format(df_class_safe_select.shape))
    
    #concat the three dataframes
    df_underSample = pd.concat([df_class_fatal, df_class_injury_select, df_class_safe_select])
    print(df_underSample.shape)
    
    #TBD
    if sampleN < df_underSample.index.size:
        df_sample = df_underSample.sample(n=sampleN)
    else:
        df_sample = df_underSample.sample(n=df_underSample.index.size)
    
    #perform the conversion in two steps to avoid any unwanted side effects
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'2'})
    print((df_sample['P_ISEV']=='0').sum())
    print((df_sample['P_ISEV']=='1').sum())
    print((df_sample['P_ISEV']=='2').sum())
    print(df_sample['P_ISEV'].unique())
else:
    df_class = df_cat.copy()

    #perform the conversion in two steps to avoid any unwanted side effects
    df_class['P_ISEV'] = df_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_class['P_ISEV'] = df_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
    print((df_class['P_ISEV']=='0').sum())
    print((df_class['P_ISEV']=='1').sum())
    print(df_class['P_ISEV'].unique())
    
    df_sample = df_class.sample(n=sampleN)

print("Size of dataframe for modeling: {}".format(df_sample.index.size))

Number of Fatal: 43559
    C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
47    1999      1      1     23      1      4      2      1      1      4   
151   1999      1      1     12      1      1      2      1      1      1   

     ...   V_ID V_TYPE V_YEAR P_ID P_SEX P_AGE P_PSN P_SAFE P_USER P_ISEV  
47   ...      1      6   1997    1     1    44    11      2      1      3  
151  ...     99      1   2017    1     1    15    99      2      3      3  

[2 rows x 21 columns]
Number of Injury: 3351616
  C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
2   1999      1      1     20      2     34      2      1      5      3   
4   1999      1      1      8      1      1      2      5      3      6   

   ...   V_ID V_TYPE V_YEAR P_ID P_SEX P_AGE P_PSN P_SAFE P_USER P_ISEV  
2  ...      2      1   1987    2     0    20    13      2      2      2  
4  ...     99      1   2017    1     1     5    99      2      3      2  

[2 rows x 21 column

In [12]:
print(df_sample[df_sample.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


## Split Training and Testing for Binary class

In [13]:
#Split between data and class
Y = df_sample[df_sample.columns[-1]]
X = df_sample[df_sample.columns[0:df_sample.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [14]:
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [15]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [16]:
#lets write the datafile for future use
df_sample.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [17]:
if enable_kmeans:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=100, tol=1e-04, verbose= verbose_level)
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Sun Nov 11 14:49:09 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 3557244.3270334112
start iteration
done sorting
end inner loop
Iteration 1, inertia 1438155.1824569283
start iteration
done sorting
end inner loop
Iteration 2, inertia 1255652.1626794364
start iteration
done sorting
end inner loop
Iteration 3, inertia 1236503.6056553617
start iteration
done sorting
end inner loop
Iteration 4, inertia 1236189.9994617817
start iteration
done sorting
end inner loop
Iteration 5, inertia 1236049.4758006847
start iteration
done sorting
end inner loop
Iteration 6, inertia 1235973.482263704
start iteration
done sorting
end inner loop
Iteration 7, inertia 1235962.4750990884
center shift 8.675758e-02 within tolerance 1.059320e-02
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 3596982.8179497942
start iteration
done sorting
end inner loop
Iteration 1, inertia

## Feature selection using Random Forest

### Feature Selection 

### Get the inportant features from random forest

In [18]:
if enable_rf_features:
    print("Random Forest Feature Selection: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
    print("Random Forest Feature Selection: Fit Start")
    forest.fit(X, Y)
    print("Random Forest Feature Selection: Fit")

    importFeatures = forest.feature_importances_
    print("Random Forest Feature Selection: Feature Importance")
    print(importFeatures)
    
    indices = np.argsort(importFeatures)[::-1]
    print(indices)
    featureLabel = X.columns[0:]
    print(featureLabel)
    rankedFeature = []
    for f in range(X.shape[1]):
        rankedFeature.append(featureLabel[indices[f]])
        print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
    print(rankedFeature)

Random Forest Feature Selection: Start
Sun Nov 11 14:49:09 2018
Random Forest Feature Selection: Fit Start
building tree 1 of 250building tree 2 of 250

building tree 3 of 250
building tree 4 of 250
building tree 5 of 250
building tree 6 of 250
building tree 7 of 250building tree 8 of 250

building tree 9 of 250
building tree 10 of 250
building tree 11 of 250
building tree 12 of 250
building tree 13 of 250building tree 14 of 250
building tree 15 of 250

building tree 16 of 250
building tree 17 of 250
building tree 18 of 250
building tree 19 of 250
building tree 20 of 250
building tree 21 of 250
building tree 22 of 250
building tree 23 of 250
building tree 24 of 250
building tree 25 of 250building tree 26 of 250building tree 27 of 250


building tree 28 of 250
building tree 29 of 250
building tree 30 of 250building tree 31 of 250

building tree 32 of 250
building tree 33 of 250
building tree 34 of 250building tree 35 of 250
building tree 36 of 250
building tree 37 of 250

building tree 

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:    0.1s


building tree 212 of 250building tree 213 of 250building tree 214 of 250building tree 215 of 250

building tree 216 of 250


building tree 217 of 250
building tree 218 of 250building tree 219 of 250

building tree 220 of 250building tree 221 of 250building tree 222 of 250

building tree 223 of 250

building tree 224 of 250
building tree 225 of 250
building tree 226 of 250building tree 227 of 250
building tree 228 of 250

building tree 229 of 250
building tree 230 of 250
building tree 231 of 250
building tree 232 of 250
building tree 233 of 250
building tree 234 of 250building tree 235 of 250

building tree 236 of 250
building tree 237 of 250
building tree 238 of 250building tree 239 of 250

building tree 240 of 250
building tree 241 of 250
building tree 242 of 250
building tree 243 of 250
building tree 244 of 250
building tree 245 of 250
building tree 246 of 250
building tree 247 of 250
building tree 248 of 250
building tree 249 of 250
building tree 250 of 250
Random Forest Feature Sel

[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.1s finished


Random Forest Feature Selection: Feature Importance
[0.07843242 0.06826468 0.05651582 0.08325091 0.03499562 0.0899008
 0.03378724 0.02975744 0.02865096 0.03811772 0.03672251 0.03533453
 0.02663645 0.0851479  0.02266789 0.02548397 0.10898781 0.03370827
 0.04883201 0.03480504]
[16  5 13  3  0  1  2 18  9 10 11  4 19  6 17  7  8 12 15 14]
Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER'],
      dtype='object')
 1) P_AGE                          0.108988
 2) C_CONF                         0.089901
 3) V_YEAR                         0.085148
 4) C_HOUR                         0.083251
 5) C_YEAR                         0.078432
 6) C_MNTH                         0.068265
 7) C_WDAY                         0.056516
 8) P_SAFE                         0.048832
 9) C_RALN                         0.038118
10) C_TRAF                   

### Reduce the number of features

In [19]:
#select features that contribute more than 0.05
if enable_rf_features:
    X_Selected = X[rankedFeature[0:10]]
else:
    if feature_all:
        X_Selected = X
    else:
        X_Selected = X[defaultFeatures]

if pyscript:
    print(X_Selected)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

print("Random Forest Feature Selection: End")


        P_AGE C_CONF V_YEAR C_HOUR C_YEAR C_MNTH C_WDAY P_SAFE C_RALN C_TRAF
2026199    57     21   1998     20   2004      4      7      2      1     18
4747143    54      4   2010     19   2012      9      7      2      2     18
2224428    70     35   1998      8   2004     11      2      1      1      4
4184139    38     21   2004     16   2010     11      2      2      1      1
1228333    87     35   2001      9   2002      3      4      2      1     18
3993623    29      2   2002      9   2010      4      2      2      3     18
853769     19      6   2001      0   2001      3      5      2      1     18
381698     54     23   1994     14   1999     12      6      2      1     18
3519943    22      6   2007      2   2008      9      1      2      2     18
2185994    20      6   1992      2   2004     10      1      1      1     18
4078120    49      3   2005     15   2010      7      4      1      1     18
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...

In [20]:
print("Split Test and Train based on Selected Features")
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [21]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [22]:
if enable_logistic_regression:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression: End")

Logistic Regression: Start
Sun Nov 11 14:49:09 2018
Logistic Regression: Fit
[LibLinear]Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.48
Accuracy of logistic regression classifier on test set: 0.42
Logistic Regression: Intercept
[ 0.00025833  0.00044898 -0.00062032]
Logistic Regression: Coefficients
[[-0.00755976  0.01528061 -0.01332646  0.01807329  0.01321654  0.03430159
  -0.0081958  -0.09038045 -0.37187294 -0.02735422]
 [-0.00920475  0.00407754  0.01666505  0.00820617 -0.01659741 -0.01581872
  -0.0115551   0.00808118 -0.15048614 -0.02005051]
 [ 0.01864343 -0.01963716 -0.0044911  -0.02945337  0.00339195 -0.01931243
   0.01869046  0.07097681  0.43358791  0.05944838]]
Logistic Regression: Confusion Matrix
[[ 99  36  61]
 [ 84  31  88]
 [ 46  30 125]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.43      0.51      0.47       196
          1       0.32      0.15      0.21     

### Logistic Regression with L1 Regularization

In [23]:
if enable_logistic_regression_l1:
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Sun Nov 11 14:49:09 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.47
Accuracy of logistic regression classifier on test set: 0.42
Logistic Regression with L1 Regularization: Confusion Matrix
[[ 97  34  65]
 [ 84  30  89]
 [ 48  28 125]]
             precision    recall  f1-score   support

          0       0.42      0.49      0.46       196
          1       0.33      0.15      0.20       203
          2       0.45      0.62      0.52       201

avg / total       0.40      0.42      0.39       600

Logistic Regression with L1 Regularization: Classification Report
Sun Nov 11 14:49:13 2018
Logistic Regression with L1 Regularization: End


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Decision Tree

In [24]:
if enable_decision_tree:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50, random_state=0)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Sun Nov 11 14:49:13 2018
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.42
Decision Tree: Confusion Matrix
[[ 71  95  30]
 [ 77  68  58]
 [ 35  51 115]]
Decision Tree: Classification Report
             precision    recall  f1-score   support

          0       0.39      0.36      0.37       196
          1       0.32      0.33      0.33       203
          2       0.57      0.57      0.57       201

avg / total       0.42      0.42      0.42       600

Sun Nov 11 14:49:13 2018
Decision Tree: End


### K-N-N

In [25]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
    print("KNN: Fit")
    knn.fit(X_train, Y_train)
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

KNN: Start
Sun Nov 11 14:49:13 2018
KNN: Fit
KNN: Predict
Accuracy of KNN classifier on train set: 0.64
Accuracy of KNN classifier on test set: 0.43
KNN: Confusion Matrix
[[103  72  21]
 [ 87  68  48]
 [ 61  52  88]]
KNN: Classification Report
             precision    recall  f1-score   support

          0       0.41      0.53      0.46       196
          1       0.35      0.33      0.34       203
          2       0.56      0.44      0.49       201

avg / total       0.44      0.43      0.43       600

Sun Nov 11 14:49:13 2018
KNN: End


### SVM

In [26]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    #svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    svm = SVC(C=1, verbose = verbose_level)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)
    
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Sun Nov 11 14:49:13 2018
SVM: Fit
[LibSVM]SVM: Predict
Accuracy of SVM classifier on train set: 1.00
Accuracy of SVM classifier on test set: 0.36
SVM: Confusion Matrix
[[ 39 139  18]
 [ 32 149  22]
 [ 14 158  29]]
SVM: Classfication Report
             precision    recall  f1-score   support

          0       0.46      0.20      0.28       196
          1       0.33      0.73      0.46       203
          2       0.42      0.14      0.21       201

avg / total       0.40      0.36      0.32       600

Sun Nov 11 14:49:14 2018
SVM: End


## ANN - Multilayer Perceptron

In [27]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=100, verbose=verbose_level)
    
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

Multilayer Preceptron: Start
Sun Nov 11 14:49:14 2018
Multilayer Preceptron: fit
Iteration 1, loss = 15.42553865
Iteration 2, loss = 13.71725614
Iteration 3, loss = 12.22575614
Iteration 4, loss = 10.08290247
Iteration 5, loss = 9.59670727
Iteration 6, loss = 8.98710167
Iteration 7, loss = 8.51382777
Iteration 8, loss = 11.46911419
Iteration 9, loss = 10.35655079
Iteration 10, loss = 8.65055746
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Multilayer Preceptron: Predict
Accuracy of Multilayer Perceptron classifier on train set: 0.33
Accuracy of Multilayer Perceptron classifier on test set: 0.33
Multilayer Preceptron: Confusion Matrix
[[196   0   0]
 [203   0   0]
 [201   0   0]]
Multilayer Preceptron: Classificiation Report
             precision    recall  f1-score   support

          0       0.33      1.00      0.49       196
          1       0.00      0.00      0.00       203
          2       0.00      0.00      0.00       201

avg / t

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Random forest

In [28]:
if enable_rf:
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0, n_jobs=2, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

Ensemble (Bagging): Random Forest: Start
Sun Nov 11 14:49:14 2018
Ensemble (Bagging): Random Forest: Fit
building tree 1 of 50building tree 2 of 50

building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50building tree 8 of 50

building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
buildin

[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


Accuracy of RandomForest classifier on train set: 1.00
Accuracy of RandomForest classifier on test set: 0.49
Ensemble (Bagging): Random Forest: Confusion Matrix
[[107  66  23]
 [ 84  54  65]
 [ 33  35 133]]
Ensemble (Bagging): Random Forest: Classification Report
             precision    recall  f1-score   support

          0       0.48      0.55      0.51       196
          1       0.35      0.27      0.30       203
          2       0.60      0.66      0.63       201

avg / total       0.48      0.49      0.48       600

Sun Nov 11 14:49:14 2018
Ensemble (Bagging): Random Forest: End


In [29]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [30]:
#clf.fit(X_train, Y_train)

In [31]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

Sun Nov 11 14:49:14 2018


In [32]:
#sys.stdout.close()
%logstop